# Raphtory Python Client - LOTR DEMO

In this demo we will
* Create a Python Raphtory client
* Create a Graph from data in Raphtory
* Pull results from an algorithm in Raphtory
* Add these results to the Graph
* Visualise the graph with pymotif

### Setup Code

First we setup the various libraries we will need

In [1]:
pip install raphtory-client

In [2]:
from raphtoryclient import raphtoryclient
from pymotif import Motif

### Create Client

Now we create a client, and then create the readers which read from topics.

If the connection fails, the code with automatically retry.

This can occur when you have not closed previous pulsar connections.

In this case we are reading the topics: EdgeList, PageRank

Note: Prior to this you should have already run these algorithms in Raphtory.

In [3]:
raphtoryClient = raphtoryclient()

In [4]:
edgeListReader = raphtoryClient.createReader("EdgeList", subscription_name='edgelist_reader')
pageRankReader = raphtoryClient.createReader("PageRank", subscription_name='pagerank_reader')

### Obtain dataframes

Now we will run the getDataframe function to retrieve results as dataframes.

In [5]:
df_edge = raphtoryClient.getDataframe(edgeListReader)
df_page = raphtoryClient.getDataframe(pageRankReader)

In [9]:
df_edge

In [10]:
df_page

### Create a graph

Next we create a graph by pulling the edge list from Raphtory.

In this case we would like to create a graph from the LOTR dataset.

So we run the `createGraphFromEdgeList` method on the `EdgeList` dataframe.

In [6]:
G = raphtoryClient.createGraphFromEdgeList(df_edge, isMultiGraph=False)
G.number_of_nodes(), G.number_of_edges()

### Adding properties to our graph
Now we merge these as node properties into our graph

In [7]:
raphtoryClient.add_node_attributes(G, [df_page], ['PageRank'])

### Visualisation

Finally we plot the graph with an open source visualisation tool .

In [8]:
motif_nx = Motif(nx_graph=G, title='NetworkX')
motif_nx.plot()